In [2]:
import os
from collections import defaultdict 

targetDir = r'C:\Users\Public\PartIIB project 2023_2024\Image collection without reaction\00AgNO3_mole_fraction\Outputs_Grayscale_Labelled_Images_Sizes\size_folder3'

def obtainSizeRange(targetDir, desiredInputNumber):
    lstDir = os.listdir(targetDir)    
    sizeDict = defaultdict(int)
    for i in lstDir:
        t_number = int(i.split("t-")[1].split("_")[0])
        lowerT = 200 - (200/desiredInputNumber) + 1
        if lowerT <= t_number <= 200:
            size = float(i[-17:-5])
            sizeRange = int(size*2)/2
            sizeDict[sizeRange] += 1

    print(sizeDict)
    print(dict(sorted(sizeDict.items())))
    print(dict(sorted(sizeDict.items(), key=lambda item: item[1])))

obtainSizeRange(targetDir, 4)

defaultdict(<class 'int'>, {7.0: 231, 8.0: 406, 7.5: 356, 13.0: 234, 12.5: 155, 24.0: 13, 24.5: 27, 25.0: 27, 4.5: 25, 5.0: 139, 32.0: 5, 31.5: 7, 31.0: 5, 30.5: 8, 30.0: 4, 29.5: 6, 29.0: 3, 28.5: 5, 28.0: 6, 27.5: 7, 27.0: 8, 26.5: 10, 26.0: 5, 25.5: 7, 23.5: 20, 23.0: 21, 22.5: 19, 22.0: 32, 21.5: 17, 21.0: 14, 33.5: 4, 33.0: 6, 32.5: 7, 17.5: 43, 17.0: 62, 16.5: 76, 16.0: 42, 15.5: 35, 12.0: 130, 11.5: 118, 11.0: 167, 10.5: 166, 10.0: 220, 20.5: 23, 20.0: 38, 19.5: 47, 19.0: 37, 18.5: 99, 18.0: 60, 15.0: 37, 14.5: 108, 14.0: 142, 6.0: 82, 6.5: 268, 8.5: 230, 9.5: 288, 9.0: 222, 2.0: 131, 5.5: 94, 2.5: 165, 4.0: 21, 1.0: 200, 13.5: 148, 0.5: 50, 3.5: 8, 3.0: 4})
{0.5: 50, 1.0: 200, 2.0: 131, 2.5: 165, 3.0: 4, 3.5: 8, 4.0: 21, 4.5: 25, 5.0: 139, 5.5: 94, 6.0: 82, 6.5: 268, 7.0: 231, 7.5: 356, 8.0: 406, 8.5: 230, 9.0: 222, 9.5: 288, 10.0: 220, 10.5: 166, 11.0: 167, 11.5: 118, 12.0: 130, 12.5: 155, 13.0: 234, 13.5: 148, 14.0: 142, 14.5: 108, 15.0: 37, 15.5: 35, 16.0: 42, 16.5: 76, 17.0